# 013-LCSM

# 課題のコンセプト
- 与えられた複数のDNA配列から、最長共通部分配列を見つける

# 課題の概要
- input: 複数の1kb未満DNA配列, FASTA形式で与えられる
- output: 与えられたすべての配列が持つ部分配列のうち、最長の部分配列

In [ ]:
import os

os.getcwd()

In [ ]:
from pathlib import Path


def read_fasta(fasta: str | Path) -> dict[str, str]:
    """FASTAファイルを読み込み、{ヘッダー: 配列} の辞書を返す"""
    fasta_dict: dict[str, str] = {}
    header = None
    sequences: list[str] = []

    for line in Path(fasta).read_text().splitlines():
        if line.startswith(">"):
            # 既存のシーケンスを保存
            if header is not None:
                fasta_dict[header] = "".join(sequences)
            # 新しいヘッダー開始
            header = line[1:].strip()
            sequences = []
        else:
            sequences.append(line)

    fasta_dict[header] = "".join(sequences)

    return fasta_dict

## 二分探索を用いた解法 (idea from 松本さん)

この解法では、**「共通部分文字列の長さ L」**を二分探索で求めます。  

### アルゴリズムの流れ

1. **候補長さ L を決める**  
   例: L=4 のとき「全ての文字列に長さ4の共通部分文字列が存在するか？」を調べます。

2. **存在判定（真偽値）**  
   - L=4 で共通部分文字列が見つかれば、  
     L=3, L=2, ... といった短い長さでも必ず存在します。
   - L=4 で見つからなければ、  
     L=5, L=6, ... のような長い長さでは絶対に存在しません。

3. **二分探索**  
   - この単調性を利用し、L を **0 〜 最短文字列の長さ** の範囲で絞り込みます。
   - L が存在する場合は右側（長い方）を探索し、存在しない場合は左側（短い方）を探索します。


In [ ]:
def substrings_of_len(s: str, L: int) -> set:
    # return all substrings of length L as a set
    if L == 0:
        return {""}
    if L > len(s):
        return set()
    return {s[i : i + L] for i in range(len(s) - L + 1)}


def longest_common_substring(sequences: list[str]) -> str:
    # use the shortest string as the base to reduce candidates
    base = min(sequences, key=len)
    others = [s for s in sequences if s is not base]

    lo, hi = 0, len(base)  # search L in [lo..hi]
    while lo <= hi:
        mid = (lo + hi) // 2
        candidates = substrings_of_len(base, mid)

        for s in others:
            candidates &= substrings_of_len(s, mid)

        if candidates:
            # length mid exists; keep any candidate and try longer
            shared_motifs = candidates
            lo = mid + 1
        else:
            hi = mid - 1
    return shared_motifs

In [ ]:
fasta_file = "example.fasta"
sequences = [seq for seq in read_fasta(fasta_file).values() if seq]
print(sequences)

In [ ]:
motifs = longest_common_substring(sequences)
print(next(iter(motifs)))

In [ ]:
fasta_file = "rosalind_lcsm.txt"
sequences = [seq for seq in read_fasta(fasta_file).values() if seq]
motifs = longest_common_substring(sequences)
print(next(iter(motifs)))